### Task 1: Data Pipeline Development

Goal: Automate the ETL (Extract, Transform, Load) process using Pandas and Scikit-learn.

Steps:

Load data from a CSV, database, or API.

Clean and preprocess it (handle missing values, outliers, normalization).

Apply transformations like feature scaling or encoding.

Store the cleaned data in a database or a new file.

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [3]:
df = pd.read_csv('Indian_Traffic_Violations.csv')
df.head(5)

,Violation_ID,Violation_Type,Fine_Amount,Location,Date,Time,Vehicle_Type,Vehicle_Color,Vehicle_Model_Year,Registration_State,...,Speed_Limit,Recorded_Speed,Alcohol_Level,Breathalyzer_Result,Towed,Fine_Paid,Payment_Method,Court_Appearance_Required,Previous_Violations,Comments
0,VLT100000,Overloading,4544,Karnataka,2023-01-01,23:02,Car,Red,2012,West Bengal,...,100,95,0.03,Negative,Yes,No,Online,Yes,3,Repeat Offender
1,VLT100001,Driving Without License,2776,Punjab,2023-01-02,00:42,Scooter,Silver,2010,Tamil Nadu,...,40,48,0.45,Negative,Yes,Yes,Online,No,2,Repeat Offender
2,VLT100002,Using Mobile Phone,4785,Maharashtra,2023-01-03,04:32,Scooter,Grey,2006,Tamil Nadu,...,80,26,0.31,Not Conducted,No,No,Not Paid,Yes,4,NaN
3,VLT100003,No Seatbelt,1138,Uttar Pradesh,2023-01-04,15:06,Car,Green,1996,Uttar Pradesh,...,100,115,0.09,Not Conducted,No,Yes,Online,No,5,Repeat Offender
4,VLT100004,Over-speeding,1610,Karnataka,2023-01-05,06:57,Truck,Yellow,2016,Delhi,...,30,115,0.28,Positive,No,Yes,Cash,Yes,0,NaN


In [4]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

df_processed = preprocessor.fit_transform(df)

processed_columns = (
    list(numeric_features) + 
    list(preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(categorical_features))
)

df_processed = pd.DataFrame(df_processed, columns=processed_columns)

df_processed.to_csv("processed_data.csv", index=False)

print("Data Preprocessing Completed and Saved Correctly!")


Data Preprocessing Completed and Saved Correctly!


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
df = pd.read_csv("processed_data.csv")
df.head(5)

,Fine_Amount,Vehicle_Model_Year,Driver_Age,Penalty_Points,Number_of_Passengers,Speed_Limit,Recorded_Speed,Alcohol_Level,Previous_Violations,Violation_ID_VLT100000,...,Fine_Paid_Yes,Payment_Method_Card,Payment_Method_Cash,Payment_Method_Not Paid,Payment_Method_Online,Court_Appearance_Required_No,Court_Appearance_Required_Yes,Comments_Fine Paid On Spot,Comments_First Violation,Comments_Repeat Offender
0,1.421200,0.370792,-1.284293,0.012512,0.708527,1.700464,0.868328,-1.540486,0.304884,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.173703,0.130758,-0.869202,-0.304240,0.708527,-0.838647,-0.730803,1.381746,-0.278626,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,1.591249,-0.349309,1.206256,0.962767,1.417408,0.854094,-1.479332,0.407669,0.888394,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,-0.982066,-1.549477,-0.039019,-0.620992,-0.709236,1.700464,1.548809,-1.123024,1.471905,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,-0.649024,0.850859,0.969061,-0.304240,0.708527,-1.261833,1.548809,0.198938,-1.445647,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [7]:
X = df.drop(columns=["Fine_Amount"]) 
y = df["Fine_Amount"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [10]:
y_pred = model.predict(X_test)


In [11]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [12]:
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²) Score: {r2}")

Mean Squared Error (MSE): 1.1614903764711961
R-squared (R²) Score: -0.13740601392611573


In [13]:
df.isnull().sum() 
df.describe()

,Fine_Amount,Vehicle_Model_Year,Driver_Age,Penalty_Points,Number_of_Passengers,Speed_Limit,Recorded_Speed,Alcohol_Level,Previous_Violations,Violation_ID_VLT100000,...,Fine_Paid_Yes,Payment_Method_Card,Payment_Method_Cash,Payment_Method_Not Paid,Payment_Method_Online,Court_Appearance_Required_No,Court_Appearance_Required_Yes,Comments_Fine Paid On Spot,Comments_First Violation,Comments_Repeat Offender
count,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4.000000e+03,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.019807e-17,5.286438e-15,-8.171241e-17,7.993606e-17,-5.595524e-17,-1.232348e-16,2.131628e-17,-5.373479e-17,3.907985e-17,0.000250,...,0.493000,0.254500,0.258250,0.247500,0.239750,0.499250,0.500750,0.505500,0.251750,0.242750
std,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,1.000125e+00,0.015811,...,0.500014,0.435634,0.437727,0.431614,0.426984,0.500062,0.500062,0.500032,0.434073,0.428799
min,-1.714476e+00,-1.669494e+00,-1.699385e+00,-1.571247e+00,-1.418117e+00,-1.261833e+00,-1.683476e+00,-1.749217e+00,-1.445647e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.813416e-01,-8.293762e-01,-8.692017e-01,-9.377437e-01,-7.092357e-01,-8.386474e-01,-9.009227e-01,-8.447164e-01,-8.621364e-01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.680822e-02,1.074150e-02,2.028019e-02,1.251170e-02,-3.544406e-04,7.723131e-03,1.772653e-02,-9.792956e-03,-2.786261e-01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,8.489600e-01,8.508592e-01,8.504633e-01,9.627671e-01,7.085268e-01,8.540936e-01,8.683277e-01,8.947074e-01,8.883944e-01,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.742952e+00,1.690977e+00,1.680646e+00,1.596271e+00,1.417408e+00,1.700464e+00,1.718929e+00,1.729631e+00,1.471905e+00,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
